In [38]:
import camelot
import pandas as pd

In [54]:
def merge_multipage_table_to_csv(
    pdf_path,
    pages="all",
    flavor="lattice",
    output_file="merged_table.csv",
    keep_header_from_first_page=True,
):
    """
    Parse a table that spans multiple pages and merge into a single CSV file

    Parameters:
    -----------
    pdf_path : str
        Path to the PDF file
    pages : str
        Pages to parse. Can be 'all', '1,2,3', or '1-3'
    flavor : str
        'lattice' for tables with lines, 'stream' for tables without lines
    output_file : str
        Name of the output CSV file
    keep_header_from_first_page : bool
        If True, uses header from first page only and skips headers on subsequent pages

    Returns:
    --------
    pandas.DataFrame: The merged table
    """

    try:
        # Read PDF tables
        print(f"Reading tables from {pdf_path}...")
        tables = camelot.read_pdf(
            pdf_path,
            pages=pages,
            flavor=flavor,
            table_areas=["28,813,567,27"],
        )

        if len(tables) == 0:
            print("No tables found in the PDF")
            return None

        print(f"Found {len(tables)} table(s) across pages")

        # Initialize merged dataframe
        merged_df = None

        for i, table in enumerate(tables):
            df = table.df

            df = df.applymap(
                lambda x: (
                    str(x).replace("\n", " ").replace("\r", " ").strip()
                    if pd.notna(x)
                    else x
                )
            )

            print(f"  Page table {i+1}: Shape {df.shape}")

            if i == 0:
                # First table - keep as is
                merged_df = df.iloc[2:]
            else:
                df = df.iloc[1:]  # Skip the header row
                merged_df = pd.concat([merged_df, df], ignore_index=True)

        # Clean up the merged dataframe
        if keep_header_from_first_page and not merged_df.empty:
            # Check if the first row looks like headers (you can customize this logic)
            first_row = merged_df.iloc[0].astype(str)
            if all(first_row.str.len() > 0):  # Basic check for non-empty headers
                merged_df.columns = first_row
                merged_df = merged_df[1:].reset_index(drop=True)
                print("\nSet first row as column headers")

        # Remove any completely empty rows
        merged_df = merged_df.dropna(how="all").reset_index(drop=True)

        # Save to CSV
        merged_df.to_csv(output_file, index=False)
        print(f"\nMerged table saved to: {output_file}")
        print(f"Final table shape: {merged_df.shape}")

        # Display preview
        print("\nPreview of merged table:")
        print(merged_df.head(10))

        return merged_df, tables

    except Exception as e:
        print(f"Error parsing PDF: {e}")
        return None

In [55]:
if __name__ == "__main__":
    df, tables = merge_multipage_table_to_csv(
        pdf_path="oficjalny_spis_pna_2025.pdf",
        pages="3-20",
        flavor="stream",
        output_file="merged_output.csv",
        keep_header_from_first_page=True,
    )

Reading tables from oficjalny_spis_pna_2025.pdf...
Found 18 table(s) across pages
  Page table 1: Shape (76, 6)
  Page table 2: Shape (81, 6)
  Page table 3: Shape (81, 6)
  Page table 4: Shape (81, 6)
  Page table 5: Shape (81, 7)
  Page table 6: Shape (81, 6)
  Page table 7: Shape (81, 6)
  Page table 8: Shape (81, 6)
  Page table 9: Shape (81, 6)
  Page table 10: Shape (81, 7)
  Page table 11: Shape (81, 6)
  Page table 12: Shape (81, 6)
  Page table 13: Shape (81, 6)
  Page table 14: Shape (81, 7)
  Page table 15: Shape (81, 7)
  Page table 16: Shape (81, 6)
  Page table 17: Shape (81, 6)
  Page table 18: Shape (81, 7)

Set first row as column headers

Merged table saved to: merged_output.csv
Final table shape: (1433, 7)

Preview of merged table:
0     PNA         Miejscowość Ulica Numery      Gmina        Powiat  \
0  83-440                  Abisynia            Karsin    kościerski   
1  20-388      Abramowice Kościelne             Głusk      lubelski   
2  20-388       Abramowice

/var/folders/g0/c58bxqzn6djcv8sqkxtcj8dc0000gn/T/ipykernel_67844/2757065500.py:51: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(
